In [1]:
import numpy as np
from keras_transformer import get_model

# Build a small toy token dictionary
tokens = 'all work and no play makes jack a dull boy'.split(' ')
token_dict = {
    '<PAD>': 0,
    '<START>': 1,
    '<END>': 2,
}
for token in tokens:
    if token not in token_dict:
        token_dict[token] = len(token_dict)

token_dict_rev = {v: k for k, v in token_dict.items()}
        
# Generate toy data
encoder_inputs_no_padding = []
encoder_inputs, decoder_inputs, decoder_outputs = [], [], []
for i in range(1, len(tokens) - 1):
    encode_tokens, decode_tokens = tokens[:i], tokens[i:]
    encode_tokens = ['<START>'] + encode_tokens + ['<END>'] + ['<PAD>'] * (len(tokens) - len(encode_tokens))
    output_tokens = decode_tokens + ['<END>', '<PAD>'] + ['<PAD>'] * (len(tokens) - len(decode_tokens))
    decode_tokens = ['<START>'] + decode_tokens + ['<END>'] + ['<PAD>'] * (len(tokens) - len(decode_tokens))
    encode_tokens = list(map(lambda x: token_dict[x], encode_tokens))
    decode_tokens = list(map(lambda x: token_dict[x], decode_tokens))
    output_tokens = list(map(lambda x: [token_dict[x]], output_tokens))
    encoder_inputs_no_padding.append(encode_tokens[:i + 2])
    encoder_inputs.append(encode_tokens)
    decoder_inputs.append(decode_tokens)
    decoder_outputs.append(output_tokens)
    print(f"encode_tokens: {[token_dict_rev[i] for i in encode_tokens]}\n decode_tokens: {[token_dict_rev[i] for i in decode_tokens]}\n output_tokens:{[token_dict_rev[i[0]] for i in output_tokens]}")
    


Using TensorFlow backend.


encode_tokens: ['<START>', 'all', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
 decode_tokens: ['<START>', 'work', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', '<END>', '<PAD>']
 output_tokens:['work', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', '<END>', '<PAD>', '<PAD>']
encode_tokens: ['<START>', 'all', 'work', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
 decode_tokens: ['<START>', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', '<END>', '<PAD>', '<PAD>']
 output_tokens:['and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', '<END>', '<PAD>', '<PAD>', '<PAD>']
encode_tokens: ['<START>', 'all', 'work', 'and', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
 decode_tokens: ['<START>', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', '<END>', '<PAD>', '<PAD>', '<PAD>']
 output_tokens:['no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', '<END>

In [2]:
print(np.asarray(decoder_outputs).shape)

(8, 12, 1)


In [3]:
# Build the model
model = get_model(
    token_num=len(token_dict),
    embed_dim=30,
    encoder_num=3,
    decoder_num=2,
    head_num=3,
    hidden_dim=120,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((13, 30)),
)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
)
model.summary()

# Train the model
print(decoder_outputs[0])
model.fit(
    x=[np.asarray(encoder_inputs * 1000), np.asarray(decoder_inputs * 1000)],
    y=np.asarray(decoder_outputs * 1000),
    epochs=5,
)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Token-Embedding (EmbeddingRet)  [(None, None, 30), ( 390         Encoder-Input[0][0]              
                                                                 Decoder-Input[0][0]              
_______________________________________________________________________________________

Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
8000/8000 [==============================] - 16s 2ms/step - loss: 0.3607
Epoch 2/5
8000/8000 [==============================] - 9s 1ms/step - loss: 0.0061
Epoch 3/5
8000/8000 [==============================] - 9s 1ms/step - loss: 0.0034
Epoch 4/5
8000/8000 [==============================] - 9s 1ms/step - loss: 0.0023
Epoch 5/5
8000/8000 [==============================] - 9s 1ms/step - loss: 0.0019


In [4]:
from keras_transformer import decode

decoded = decode(
    model,
    encoder_inputs_no_padding,
    start_token=token_dict['<START>'],
    end_token=token_dict['<END>'],
    pad_token=token_dict['<PAD>'],
    max_len=100,
)


test = np.asarray(decoder_outputs)
for i in range(len(decoded)):
    print(' '.join(map(lambda x: token_dict_rev[x], decoded[i][1:-1])))

work and no play makes jack a dull boy
and no play makes jack a dull boy
no play makes jack a dull boy
play makes jack a dull boy
makes jack a dull boy
jack a dull boy
a dull boy
dull boy


In [5]:
print(decoder_outputs)

[[[4], [5], [6], [7], [8], [9], [10], [11], [12], [2], [0], [0]], [[5], [6], [7], [8], [9], [10], [11], [12], [2], [0], [0], [0]], [[6], [7], [8], [9], [10], [11], [12], [2], [0], [0], [0], [0]], [[7], [8], [9], [10], [11], [12], [2], [0], [0], [0], [0], [0]], [[8], [9], [10], [11], [12], [2], [0], [0], [0], [0], [0], [0]], [[9], [10], [11], [12], [2], [0], [0], [0], [0], [0], [0], [0]], [[10], [11], [12], [2], [0], [0], [0], [0], [0], [0], [0], [0]], [[11], [12], [2], [0], [0], [0], [0], [0], [0], [0], [0], [0]]]


In [6]:
for i in decoder_outputs:
    for j in i:
        print([token_dict_rev[k] for k in j])

['work']
['and']
['no']
['play']
['makes']
['jack']
['a']
['dull']
['boy']
['<END>']
['<PAD>']
['<PAD>']
['and']
['no']
['play']
['makes']
['jack']
['a']
['dull']
['boy']
['<END>']
['<PAD>']
['<PAD>']
['<PAD>']
['no']
['play']
['makes']
['jack']
['a']
['dull']
['boy']
['<END>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['play']
['makes']
['jack']
['a']
['dull']
['boy']
['<END>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['makes']
['jack']
['a']
['dull']
['boy']
['<END>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['jack']
['a']
['dull']
['boy']
['<END>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['a']
['dull']
['boy']
['<END>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['dull']
['boy']
['<END>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
['<PAD>']
